In [61]:
from yahoo_finance import Share
import requests
import base64
from bs4 import BeautifulSoup
import pandas
import sqlite3 as lite
from shutil import copyfileobj
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from datetime import datetime , date ,timedelta
import sys
import os
import re
#from time import sleep
import time
import mysql.connector

#define max stock id
max_stock_id = 1000

#payload data for twse
payload={}
dt_str = []
#gmail setting
emailfrom = "gghammer@gmail.com"
emailto = "gghammer@gmail.com"
#fileToSend = "daily.xlsx"
username = "gghammer"
password = "ggpudin81161"

def conn_mysql():
    try:
        #server = '192.168.0.110' 
        server = 'ggnas.synology.me'
        cnx = mysql.connector.connect(user='stock', password='stock168',host= server,database='stock')  
        print("Connector to MYSQL Server:",server,"SUCCESS")
        return cnx 
    except OSError as err:
        print("OS error: {0}".format(err))   
        try:
            #server = 'ggnas.synology.me'
            server = '192.168.0.110'
            cnx = mysql.connector.connect(user='stock', password='stock168',host= server,database='stock')   
            print("Connector to MYSQL Server:",server,"SUCCESS")
            return cnx 
        except OSError as err:
            print("OS error: {0}".format(err))
            print("Connector to MYSQL Server FAIL") 
    
def send_to_gmail(source,attach_name):    
    #save to xlsx file
    twse = pandas.read_csv(source,encoding = "big5",header=1)    
    writer = pandas.ExcelWriter(attach_name, engine='xlsxwriter')
    twse.to_excel(writer, sheet_name='Sheet1')
    writer.save()

    #gmail send 
    msg = MIMEMultipart()
    msg["From"] = emailfrom
    msg["To"] = emailto
    today = date.today()    
    #msg["Subject"] = "哈墨投顧日報 "+currenttime.strftime('%Y-%m-%d')
    msg["Subject"] = "哈墨投顧日報 "+str(today)    
    msg.preamble = "help I cannot send an attachment to save my life"

    ctype, encoding = mimetypes.guess_type(attach_name)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "text":
        fp = open(attach_name)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "image":
        fp = open(attach_name, "rb")
        attachment = MIMEImage(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "audio":
        fp = open(attach_name, "rb")
        attachment = MIMEAudio(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(attach_name, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    attachment.add_header("Content-Disposition", "attachment", filename=attach_name)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(username,password)
    server.sendmail(emailfrom, emailto, msg.as_string())
    server.quit()
    print("Send Gmail done")

def crawl_stock_id(name):  
    #catch payload 
    res = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_d.php#')
    res.encoding = 'big5'

    #find key & Encryption 
    soup = BeautifulSoup(res.text, 'html.parser') 

    for inp in soup.select('input'):
        if 'hidden' in inp.get('type'): 
            payload[inp.get('name')] = base64.b64encode(inp.get('value').encode('utf-8')) 
        
    #download csw file        
    res2 = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_print.php?language=ch&save=csv',data = payload ,stream = True)

    #save to twse.csv 
    f = open(name,'wb')
    copyfileobj (res2.raw,f)
    f.close()  
    
    
def crawl(target):  
    
    
    #catch payload 
    res = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_d.php#')
    res.encoding = 'big5'

    #find key & Encryption 
    soup = BeautifulSoup(res.text, 'html.parser') 

    #get & translate date 
    dt = soup.select('input')[2].get('value')
    dt = dt.replace(dt[0:3], str(int(dt[0:3])+ 1911))
    dt = dt.replace(dt[4:5],'-')
    dt_str = dt.split(' ')

    for inp in soup.select('input'):
        if 'hidden' in inp.get('type'): 
            payload[inp.get('name')] = base64.b64encode(inp.get('value').encode('utf-8')) 
        
    #download csw file        
    res2 = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_print.php?language=ch&save=csv',data = payload ,stream = True)

    #save to twse.csv 
    f = open('twse.csv','wb')
    copyfileobj (res2.raw,f)
    f.close()  

    #read csw to pandas 
    twse = pandas.read_csv('twse.csv',encoding = "big5",header=1)

    with lite.connect(target) as conn:   
        for i in range(0,max_stock_id):
            col = twse.iloc[i]
            col = col.tolist()
            stock_id = col[0]
            if len(stock_id)>10 :
                break
            else:    
                data = dt_str + col
                conn.execute('INSERT INTO 指標 (日期,代號,名稱,本益比,殖利率,股價淨值比) VALUES (?,?,?,?,?,?);', data)
    conn.close()
    
    print("crawl 指標 is Done")
    
def update_stock_id(name):    

    crawl_stock_id('temp.csv')
    #read csw to pandas 
    twse = pandas.read_csv('temp.csv',encoding = "big5",header=1)
    
    conn = conn_mysql()
    cursor = conn.cursor()    
    for i in range(0,max_stock_id):
        col = twse.iloc[i]
        col = col.tolist()
        del col[2:5]   
        stock_id = col[0]
        stock_name = col[1]
        if len(stock_id)>10:
            break
        else:   
            cursor.execute ("UPDATE 股票代號 SET 代號='%s',名稱='%s' WHERE ID='%d'" % (stock_id,stock_name,i+1))
           
    conn.commit()
    conn.close()  
    
    os.remove("temp.csv")
    print("Stock ID update success !")
    
def del_sql_lib():
    conn = conn_mysql()
    cursor = conn.cursor()
    cursor.execute("DROP TABLE 股票代號")   
    cursor.execute("DROP TABLE 股價") 
    cursor.execute("CREATE TABLE 指標")
    cursor.execute("CREATE TABLE 營收")    
    
    
def create_sql_lib(name):    
    #create sqlite lib
    conn = conn_mysql()
    cursor = conn.cursor()
    
    cursor.execute("""CREATE TABLE 股票代號 (
                                            ID   INT(12) NOT NULL AUTO_INCREMENT PRIMARY KEY,
                                            代號 VARCHAR(10)CHARACTER SET utf8 COLLATE utf8_general_ci,
                                            名稱 VARCHAR(10)CHARACTER SET utf8 COLLATE utf8_general_ci )""")                  
    cursor.execute("""CREATE TABLE 股價 (
                                        ID   INT(12) NOT NULL AUTO_INCREMENT PRIMARY KEY,
                                        日期 DATE,
                                        代號 INT(12),
                                        名稱 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci, 
                                        開盤價 FLOAT,
                                        收盤價 FLOAT,
                                        漲跌 FLOAT,
                                        漲跌百分比 FLOAT,
                                        最高 FLOAT,
                                        最低 FLOAT,
                                        日量 INT(12),
                                        均線50 FLOAT,
                                        均線200 FLOAT)""")          
    
    cursor.execute("""CREATE TABLE 指標 (
                                         ID   INT(12) NOT NULL AUTO_INCREMENT PRIMARY KEY,       
                                         日期 DATE ,
                                         代號 INT(12),
                                         名稱 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci, 
                                         本益比 FLOAT,
                                         殖利率 FLOAT,
                                         股價淨值比 FLOAT)""")    
    cursor.execute("""CREATE TABLE 營收 (
                                         ID   INT(12) NOT NULL AUTO_INCREMENT PRIMARY KEY,       
                                        日期 DATE,
                                        代號 INT(12),
                                        名稱 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        EPS FLOAT,
                                        月營收 FLOAT,
                                        季營收 FLOAT,
                                        年營收 FLOAT,
                                        配息 FLOAT,
                                        配股 FLOAT)""")
    
    print ("MySQL LIB created successfully")  
    
    
    crawl_stock_id('temp.csv')
    
    #read csw to pandas 
    twse = pandas.read_csv('temp.csv',encoding = "big5",header=1)

    #update to sqlite file 
    for i in range(0,max_stock_id):
        col = twse.iloc[i]
        col = col.tolist()
        del col[2:5]
        if len(col[0]) >10:
            break
        else:    
            cursor.execute("INSERT INTO 股票代號 (代號,名稱) VALUES ('%s','%s')"%(col[0],col[1]))   
    conn.commit()           
    conn.close()      

    
    os.remove("temp.csv")
    print ("Stock ID created successfully") 
    
    
def getStock_history(id,start,end):
    stock = Share(str(id)+'.TW')
    data = stock.get_historical(start, end)
    return data


def crawl_history(target):     
    today = date.today() #todays date 
    conn = conn_mysql()
    cursor = conn.cursor(buffered = True)
    cursor.execute("SELECT * FROM 股票代號") 
    for i in range(0,max_stock_id):             
        sql = cursor.fetchone()
        print(sql)
        if sql:    
            try:
                #print(sql[2])
                stock = getStock_history(sql[1],'2008-01-01',str(today))    
                for i in range(0,len(stock)):                            
                    data = stock[i]
                    #print(data)
                    if data['Date']:
                        stock_date = data['Date']
                    if data['Open']:  
                        stock_open = data['Open']
                    if data['High']:                         
                        stock_high = data['High']
                    if data['Low']:                         
                        stock_low = data['Low']
                    if data['Close']:         
                        stock_close = data['Close']
                    if data['Volume']:
                        stock_volume = (int(data['Volume'])/1000)
                    if int(stock_volume):      
                        #sql_list = [stock_date]+[sql[0]]+[sql[1]]+[stock_open]+[stock_close]+[stock_high]+[stock_low]+[stock_volume]
                        cursor2 = conn.cursor(buffered = True)
                        cursor2.execute("INSERT INTO 股價 (日期,代號,名稱,開盤價,收盤價,最高,最低,日量) VALUES ('%s','%s','%s','%s','%s','%s','%s','%d')"%(stock_date,sql[1],sql[2],stock_open,stock_high,stock_low,stock_close,stock_volume))                     
                conn.commit()    
                print(datetime.now())  
            except OSError as err:
                print("OS error: {0}".format(err))
                print("Not support ID =",sql[0])            
    conn.close()    
    print("Crawl history is done")
    
#################個股日成交資訊 #################
def getStockCommandOneWithData(fileFolder, year, month, stockID):
    fileType = 'csv'
    payload = {
        'download':str(fileType),
        'query_year':str(year),
        'query_month':str(month),
        'CO_ID':str(stockID),
        'query-button':'查詢'
    }
        
    try:
        url = 'http://www.twse.com.tw/ch/trading/exchange/STOCK_DAY/STOCK_DAYMAIN.php'
        encodeMethod = 'big5'
            
        fileName = str(stockID) + "_" + str(year) + "_" + str(month) + "."+fileType
        savePath = fileFolder + fileName
            
            
        res = requests.post(url, data=payload, stream=True)
        f = open(savePath, 'wb')
        copyfileobj(res.raw, f)
        f.close()
            
            
            
        #file = open(savePath,"wb") #open file in binary mode
        #file.write(the_page)
        #file.close()
            
        #webpage = open(savePath, 'r')
        #twse = pandas.read_csv(savePath,encoding = "big5",header=1)
        #newspd = pandas.DataFrame(twse)
        #print(newspd)
        time.sleep(0.1)
        print ("%s:" % time.ctime(time.time()))
            
        print(str(stockID) + ' in ' + str(year) + '/' + str(month) + ' OK')
        return fileName
    except OSError as err:
        print("OS error: {0}".format(err))
        print(str(stockID) + ' Get Data Fail' + ' in ' + str(year) + '/' + str(month) )
        return None
                
    
def getStockPERAndYieldByDayWithData(fileFolder, year, month, day):
        
    url1 = 'http://www.twse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_d.php'
    url2 = 'http://www.twse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_print.php?language=ch&save=csv&save=csv&save=csv'
    nYear = year - 1911
    
    nMonth = str(month)
    nDay = str(day)
    if month < 10:
        
        nMonth = '0' + str(month)
        
    if day < 10:
        nDay = '0' + str(day)
            
    newDateStr = str(nYear)+'/'+str(nMonth)+'/'+str(nDay)
    print(newDateStr)
        
    postOneData = {
        'input_date':newDateStr,
        'select2':'ALL',
        'order':'STKNO',
        'login_btn':'(unable to decode value)'
    }
        
    try:
            
        encodeMethod = 'big5'
        payloadData = {}

        fileName = str(year) + "_" + str(month) + "_" + str(day) + "." + 'csv'
        savePath = fileFolder + fileName

        res = requests.post(url1, data=postOneData)
        res.encoding = encodeMethod
        soup = BeautifulSoup(res.text, 'html.parser')
        
        for inp in soup.select('input'):
            if 'hidden' in inp.get('type'):
                payloadData[inp.get('name')] = base64.b64encode(inp.get('value').encode('utf-8'))
        
        res2 = requests.post(url2, data=payloadData, stream=True)
        f = open(savePath, 'wb')
        copyfileobj(res2.raw, f)
        f.close()
        
        webpage = open(savePath, 'r')
        twse = pandas.read_csv(savePath,encoding = "big5", header=1, names=['證券代號', '證券名稱', '本益比', '殖利率(%)', '股價淨值比'])
        newspd = pandas.DataFrame(twse)
        
        if newspd.columns[0] != '證券代號':
            os.remove(savePath)
            print(str(year) + '/' + str(month) + '/' + str(day) + ' No Data')
            return None
        #else:
            #print ("%s:" % time.ctime(time.time()))
            #print(str(year) + '/' + str(month) + '/' + str(day) + ' OK')
        
        #print(payloadData)
            
        #webpage = open(savePath, 'r')
        #twse = pandas.read_csv(savePath,encoding = "big5",header=1)
        #newspd = pandas.DataFrame(twse)
        #print(newspd)
        #time.sleep(0.1)
        
        return fileName
    except OSError as err:
        print("OS error: {0}".format(err))
        print(' Get Data Fail' + ' in ' + str(year) + '/' + str(month) + '/' + str(day) )
        return None 
    
def crawl_本益比殖利率_by_Day(delta_days):
    conn = conn_mysql()
    cursor = conn.cursor(buffered = True)     
    currenttime = datetime.now() 
    for i in range(1,delta_days):
        dt = currenttime - timedelta( days = i)
        filename = getStockPERAndYieldByDayWithData('/Users/gghammer/Desktop/python/database/指標/',int(dt.strftime('%Y')),int(dt.strftime('%m')),int(dt.strftime('%d')))
        if filename:           
            #read csw to pandas 
            twse = pandas.read_csv('./database/指標/'+filename,encoding = "big5",header=1)  
            for i in range(0,max_stock_id):
                col = twse.iloc[i]
                col = col.tolist()
                stock_id = col[0]
                stock_pe = round(float2(col[2]),2)
                stock_dy = round(float2(col[3]),2)
                stock_pbr = round(float2(col[4]),2)
                #print(stock_pe,stock_dy,stock_pbr)
                if len(stock_id)>10 :
                    break
                else:    
                    cursor.execute("INSERT INTO 指標 (日期,代號,名稱,本益比,殖利率,股價淨值比)VALUES ('%s','%d','%s','%f','%f','%f')"
                                    %(dt.strftime('%Y-%m-%d'),int(stock_id),col[1],stock_pe,stock_dy,stock_pbr)) 
        conn.commit()
        print(datetime.now()) 
    conn.close()
    print("crawl 指標 is Done")
      
def crawl_個股_by_Month(delta_month):
    conn = conn_mysql()
    cursor = conn.cursor(buffered = True)
    cursor.execute("SELECT * FROM 股票代號") 
    #for i in range(0,max_stock_id):          
    for i in range(0,100):          
        sql = cursor.fetchone()
        print(sql)
        if sql:  
            currenttime = datetime.now() 
            for i in range(1,delta_month):
                dt = currenttime - timedelta( days = i*(365/12))
                stock_id = sql[1]
                stock_name = sql[2]
                filename = getStockCommandOneWithData('/Users/gghammer/Desktop/python/database/個股/',
                                                      int(dt.strftime('%Y')),int(dt.strftime('%m')),stock_id)
                if filename:                             
                    #read csw to pandas 
                    twse = pandas.read_csv('./database/個股/'+filename,encoding = "big5",skiprows=[0],index_col = False)  
                    try: 
                        for i in range(0,31):
                            col = twse.iloc[i]
                            col = col.tolist()
                            stock_date = col[0]
                            stock_volume = int(col[1].replace(",",""))
                            stock_open = round(float2(col[3]),2)
                            stock_high = round(float2(col[4]),2)
                            stock_low  = round(float2(col[5]),2)  
                            stock_close = round(float2(col[6]),2)
                            stock_delta = round(float2(col[7]),2)
                            stock_deal = int(col[8].replace(",",""))
                            if len(stock_id)>10:
                                break
                            else:   
                                cursor.execute("INSERT INTO 股價 (日期,代號,名稱,開盤價,收盤價,漲跌,最高,最低,日量)VALUES ('%s','%d','%s','%f','%f','%f','%f','%f','%d')"
                                               %(stock_date,int(stock_id),stock_name,stock_open,stock_close,stock_delta,stock_high,stock_low,stock_volume))
                    except:
                        print("")
            conn.commit()
            print(datetime.now()) 
    conn.close()
    print("crawl 個股 is Done")    
    
def float2(value):
    
    try:
        value = float(value)
        return value
    except:
        return 0.0    
            
#main   
def main(fucntion):
    print("gogo")    
#    if fucntion == 'create_lib':
#    create_sql_lib('stock.sqlite')     
#    del_sql_lib()    
        
#    elif fucntion == 'update_stock_id':conn.commit(
#    update_stock_id('stock.sqlite')  
    
#    elif fucntion == 'crawl':      
#        crawl('stock.sqlite')  #target sql file name , para    
#    stock = crawl_history('stock.sqlite')  
#    crawl_本益比殖利率_by_Day(2)
    crawl_個股_by_Month(2)
    #sts = getStockPERAndYieldByDayWithData('/Users/gghammer/Desktop/python/',2017,3,20)
#    elif fucntion == 'crawl_history':      
#        stock = crawl_history('stock.sqlite')   
#    elif fucntion == 'report':
#        send_to_gmail('twse.csv','daily.xlsx') #source pandas structure , target excel file name        
#    else:
#        print(' Usag   hamu create_lib          #建立資料庫')
#        print('        hamu update_stock_id     #更新股票代號')
#        print('        hamu crawl               #爬當日資料')
#        print('        hamu crawl               #爬歷史資料')
#        print('        hamu report              #透過gmail寄出電子報')
        
        
if __name__ == '__main__':
    main(sys.argv[1])     

gogo
Connector to MYSQL Server: ggnas.synology.me SUCCESS
(1, '1101', '台泥')
Fri Mar 31 18:49:27 2017:
1101 in 2017/3 OK

2017-03-31 18:49:29.938191
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
crawl 個股 is Done
